In [15]:
# Antes de importar tweepy es necesario instalar tweepy (ver la explicación en el post).
import tweepy



In [17]:
from tweepy import OAuthHandler

In [18]:
# Ahora procedemos a autenticarnos usando el API Key, el API secret Key, el Access token y el Access token secret que se generaron al crear nuestra aplicación en twitter
api_key = "TP2lHjx0czwt6JritkMZpDx2Z"
api_secret = "TsCRjuFPtzqZcWUyxyUSIb0Qc9uUdSlpuSyV5USSa38ll2OjpY"
access_token = "382368016-bKaEFl0nvbY37XWXeJI1dq7Dn7i4k8QUnmh9KcZA"
access_secret = "hK3Ozwa2YwDLENJ0Orz247jo7vh7nuLSmLAhoN6nr3oy9"
auth = OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth) 


In [12]:
import sys

class MyStreamListener(tweepy.StreamListener):

    def on_data(self, data):
        try:
            with open('t.json', 'a') as f:
                f.write((data))
                
                
                return True
        except BaseException as e:
            print("Error on_data: %s" % str(e))
        return True
 
    def on_error(self, status):
        print(status)
        return True

myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener)
myStream.filter(track=['JEP'])

KeyboardInterrupt: 

In [7]:
import re
from nltk.tokenize import word_tokenize
import json
from json import JSONDecoder
from functools import partial
from collections import defaultdict
from collections import Counter 
import operator
from string import punctuation
from nltk.corpus import stopwords
import string
from nltk import bigrams 
import numpy as np
from numpy import *
import builtins
from nltk.stem.porter import *
from nltk.stem.snowball import SnowballStemmer


stemmer = SnowballStemmer("spanish")
 
emoticons_str = r"""
    (?:
        [:=;] # :(
        [oO\-]? # No se
        [D\)\]\(\]/\\OpP] # 
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)


               

            
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens

 
def replace_iter(iterable, search, replace):
    for value in iterable:
        value.replace(search, replace)
        yield value
fname = 'p.json'
punctuation = list(string.punctuation)
stop = stopwords.words('spanish') + punctuation + ['RT', 'via', 'de', 'la','La']

    
with open('p.json', 'r') as f:
    iterable = replace_iter(f, "\\r", "")
    for line in f:
        tweet = json.loads(line)
        tokens = preprocess(tweet['text'])
    com = defaultdict(lambda : defaultdict(int))
 
    
with open(fname, 'r') as f:
    count_all = Counter()
    for line in f:
        tweet = json.loads(line)
        # crear lista con todos los terminos
        terms_all = [term for term in preprocess(tweet['text'])]
        terms_stop = [term for term in preprocess(tweet['text']) if term not in stop]
        terms_only = [term for term in preprocess(tweet['text']) if term not in stop and
                        not term.startswith(('#', '@'))] 
        lem = [stemmer.stem(term) for term in terms_only] #lematizacion 
        
        print(lem)
        count_all.update(terms_only)
        for i in range(len(terms_only)-1):            
            for j in range(i+1, len(terms_only)):
                w1, w2 = sorted([terms_only[i], terms_only[j]])                
                if w1 != w2:
                    com[w1][w2] += 1
        # actualizar contador
        
    print("Imprimir n terminos mas frecuentes")
    print(count_all.most_common(100)) 
    
terms_bigram = bigrams(terms_only)
com_max = []
# para cada termino ver los mas concurrentes terminos
for t1 in com:
    t1_max_terms = sorted(com[t1].items(), key=operator.itemgetter(1), reverse=True)[:10]
    for t2, t2_count in t1_max_terms:
        com_max.append(((t1, t2), t2_count))
# obtener co-ocurrencias
terms_max = sorted(com_max, key=operator.itemgetter(1), reverse=True)
print(terms_max[:10])


with open(fname, 'r') as f:
    search_word = 'JEP' # pasar un termino para busqueda
    count_search = Counter()

    for line in f:
        tweet = json.loads(line)
        terms_only = [term for term in preprocess(tweet['text']) 
                      if term not in stop 
                      and not term.startswith(('#', '@'))]
        if search_word in terms_only:
            count_search.update(terms_only)
    print("Co-occurrence for %s:" % search_word)
    print(count_search.most_common(20))






p_t = {}
p_t_com = defaultdict(lambda : defaultdict(int))
n_docs = 808
 
for term, n in count_all.items():
    p_t[term] = n / n_docs
    for t2 in com[term]:
        p_t_com[term][t2] = com[term][t2] / n_docs

positive_vocab = [
    'respetar', 'paz', 'nomas', 'guerra', 'exguerrillero', 'por', 'fin', 'alfin'
    'grandioso', 'genial', ':)', ':-)', 'bueno', 'felices', 'triunfo', 'acuerdos', '', 'victoria', 'defender',
    'valores'
]
negative_vocab = [
    'guerrillero', 'paraco', 'terrorista', 'ñero', 'impunidad', ':(', ':-(', 'uribestias', 'uribestia', 'inservibles',
    'marioneta', 'titere', 'narcotraficantes', 'narcotraficante', "carcel", 'perdidos','petroñero',
    'narcoterrorismo'
]

pmi = defaultdict(lambda : defaultdict(int))
for t1 in p_t:
    for t2 in com[t1]:
        denom = p_t[t1] * p_t[t2]
        pmi[t1][t2] = math.log2(p_t_com[t1][t2] / denom)

semantic_orientation = {}
for term, n in p_t.items():
    positive_assoc = builtins.sum(pmi[term][tx] for tx in positive_vocab)
    negative_assoc = builtins.sum(pmi[term][tx] for tx in negative_vocab)
    semantic_orientation[term] = positive_assoc - negative_assoc
    

semantic_sorted = sorted(semantic_orientation.items(), 
                         key=operator.itemgetter(1), 
                         reverse=True)
top_pos = semantic_sorted[:10]
top_neg = semantic_sorted[-10:]

print("resultados positivos")
print(top_pos)
print("resultados negativos")
print(top_neg)

print("Uribe: %f" % semantic_orientation['Uribe'])
print("JEP: %f" % semantic_orientation['jep'])
print("titere de Uribe: %f" % semantic_orientation['Duque'])
print("Sacionar JEP: %f" % semantic_orientation['sancionar'])
print("presidente: %f" % semantic_orientation['presidente'])
print("Acuerdos de paz: %f" % semantic_orientation['acuerdos'])
print("Impunidad: %f" % semantic_orientation['impunidad'])      
    


['a', 'ver', 'senador', 'urib', 'despu', 'es', '9', 'mes', 'perd', 'derrot', 'cam', 'derrot', 'sen', 'sentenci', 'q', '…']
['en', 'enlac', 'tambi', 'en', 'pued', 'descarg', 'ley', 'estatutari', 'jep', 'https://t.co/7p1s89hium', 'https://t.co/dve6wj8sdl']
['propon', 'acab', 'jep', 'farc', ';p', 'resident', '…']
['a', 'ver', 'senador', 'urib', 'despu', 'es', '9', 'mes', 'perd', 'derrot', 'cam', 'derrot', 'sen', 'sentenci', 'q', '…']
['pte', 'sient', 'derrot', 'hab', 'sancion', 'ley', 'estatutari', 'jep', 'al', 'contrari', 'sientat', 'tranquil', '…']
['com', 'qued', 'ojit', '…']
['pte', 'sient', 'derrot', 'hab', 'sancion', 'ley', 'estatutari', 'jep', 'al', 'contrari', 'sientat', 'tranquil', '…']
['aqu', 'i', 'pued', 'ver', 'quej', 'andos', '“', 'oblig', '”', 'vot', 'bloqu', 'ascens', 'general', 'que', 'rar', '…']
['apel', 'altur', 'jep', 'sirv', 'despu', 'es', 'viaj', 'play', 'contrat', 'herman', 'linar', 'firm', 'falsific', '…']
['pte', 'sient', 'derrot', 'hab', 'sancion', 'ley', 'estatu

['asi', 'est', 'an', 'urib', 'enter', 'duqu', 'sancion', 'o', 'ley', 'estatutari', 'jep', 'este', 'sid', 'año', '…']
['mi', 'obsesi', 'on', 'adel', 'ser', 'a', 'busc', 'mecan', 'legal', 'derogaci', 'on', 'ley', 'estatutari', 'jep', 'asi', 'm', '…']
['¿', 'y', 'si', 'sancion', 'jep', 'tiemp', 'total', 'respet', 'institucional', 'acuerd', 'por', 'favor', '…']
['corrupt', '9', 'magistr', 'dieron', 'golp', 'estad', 'aprob', 'barbar', 'comun', 'enriqu', 'santi', '…']
['sig', 'soñ', 'cual', 'verd', 'victim', 'justici', 'victimari', 'ha', 'reconoc', 'farc', 'algun', '…', 'https://t.co/4ataxa7jek']
['señor', 'onu', '1', 'uds', 'sab', 'ian', 'jep', 'liber', 'ia', 'santrich', '2', 'uds', 'sab', 'armas', 'drog', 'etcr', '3', 'uds', 'sab', 'amenaz', '…']
['negoci', 'redond', 'result', 'o', 'farc', 'estaf', 'acuerd', 'paz', 'maridaj', 'altas', 'cort', '…']
['viej', 'marihuaner']
['un', 'nuev', 'pas', 'trascendental', 'afianz', 'paz', 'president', 'sancion', 'ley', 'estatutari', 'jep', '…']
['ley', 

['a', 'ver', 'senador', 'urib', 'despu', 'es', '9', 'mes', 'perd', 'derrot', 'cam', 'derrot', 'sen', 'sentenci', 'q', '…']
['¡', 'por', 'fin', 'sancion', 'ley', 'estatutari', 'jep', 'https://t.co/qtsdzw22k6', 'https://t.co/1vmzkfqpy3']
['¡', 'muy', 'import', 'decisi', 'on', 'abrir', 'participaci', 'on', 'victim', 'proces', 'permit', 'contribu', 'ampli', 'bas', 'l', '…']
['a', 'ver', 'senador', 'urib', 'despu', 'es', '9', 'mes', 'perd', 'derrot', 'cam', 'derrot', 'sen', 'sentenci', 'q', '…']
['¡', 'por', 'fin', 'sancion', 'ley', 'estatutari', 'jep', 'https://t.co/qtsdzw22k6', 'https://t.co/1vmzkfqpy3']
['y', 'ahi', 'si', 'mand', 'guevon', 'marran', 'habl', 'president']
['segur', 'hiz', 'ya', 'sali', 'o', 'urib', 'dec', 'deb', 'cre', 'tribunal', 'cort', 'dios', 'mio', 'señor', 'pid', 'l', '…', 'https://t.co/rmejstkc7']
['referend', 'comit', 'e', 'nacional', 'referend', 'colombi', '2019', 'comunic', 'no', '001', 'restauraci', '…', 'https://t.co/edrecjlfm']
['al', 'fin', 'acab', 'sancion',

['este', 'miercol', 'cort', 'constitucional', 'defin', 'a', 'futur', 'objecion', 'ley', 'estatutari', 'jep', 'https://t', '…']
['jep', 'ocult', 'o', 'santrich', 'cometi', 'o', 'delit', 'narcotr', 'afic', 'lueg', 'acuerd', 'paz', 'https://t.co/9pjh3mkj2v']
['confirm', 'salg', 'call', 'protest', 'jep', 'impun', 'averigu', 'ciud', 'med', '…']
['com', 'obvi', 'ivan', 'duqu', 'toc', 'o', 'sancion', 'ley', 'estatutari', 'jep', '👏', '🏼', '👏', '🏼', '👏', '🏼', '10', 'mes', 'perd', 'lleg', '…']
['jep', 'elu', 'buk', 'ini']
['¡', 'por', 'fin', 'sancion', 'ley', 'estatutari', 'jep', 'https://t.co/qtsdzw22k6', 'https://t.co/1vmzkfqpy3']
['un', 'nuev', 'pas', 'trascendental', 'afianz', 'paz', 'president', 'sancion', 'ley', 'estatutari', 'jep', '…']
['gan', 'pais', 'bien', 'perdi', 'uberrim', 'secuac', 'en', 'paz', 'jam', 'as', 'podr', 'an', 'https://t.co/lsksh7asrm']
['en', 'camp', 'ña', 'quer', 'ia', 'acab', 'unific', 'cort', 'ahor', 'quier', 'met', 'jep', 'hagam', 'favor', 'urib', 'busc', 'a', '…',

['señor', 'onu', '1', 'uds', 'sab', 'ian', 'jep', 'liber', 'ia', 'santrich', '2', 'uds', 'sab', 'armas', 'drog', 'etcr', '3', 'uds', 'sab', 'amenaz', '…']
['falt', 'firm', 'duqu', 'jep', 'vigenci', 'https://t.co/pjoajblkm1']
['un', 'nuev', 'pas', 'trascendental', 'afianz', 'paz', 'president', 'sancion', 'ley', 'estatutari', 'jep', '…']
['¿', 'y', 'si', 'sancion', 'jep', 'tiemp', 'total', 'respet', 'institucional', 'acuerd', 'por', 'favor', '…']
['¿', 'y', 'si', 'sancion', 'jep', 'tiemp', 'total', 'respet', 'institucional', 'acuerd', 'por', 'favor', '…']
['que', 'silenci', 'lider', 'cd', 'nunc', 'ser', 'a', 'buen']
['a', 'ver', 'senador', 'urib', 'despu', 'es', '9', 'mes', 'perd', 'derrot', 'cam', 'derrot', 'sen', 'sentenci', 'q', '…']
['ley', 'estatutari', 'jep', 'sid', 'sancion', 'este', 'triunf', 'dispuest', 'enfrent', 'horr', '…']
['y', 'com', 'explic', 'señor', 'dic', 'magistr', 'jep', 'guerri', '…', 'https://t.co/dsvc4tsuzv']
['al', 'fin', 'acab', 'sancion', 'ley', '1957', '2019',

['un', 'nuev', 'pas', 'trascendental', 'afianz', 'paz', 'president', 'sancion', 'ley', 'estatutari', 'jep', '…']
['bye', 'jep', '🐱', '🥺', 'https://t.co/ahi3vx32s']
['un', 'nuev', 'pas', 'trascendental', 'afianz', 'paz', 'president', 'sancion', 'ley', 'estatutari', 'jep', '…']
['a', 'ver', 'senador', 'urib', 'despu', 'es', '9', 'mes', 'perd', 'derrot', 'cam', 'derrot', 'sen', 'sentenci', 'q', '…']
['colombi', 'pued', 'naufrag', 'injustici', 'impun', 'imposici', 'on', 'farian', 'santist', 'est', 'a', 'busc', 'destru', 'nuestr', '…']
['los', 'cas', 'ser', 'an', 'revis', 'colombi', 'calif', 'terc', 'punt', 'much', 'impun', '…', 'https://t.co/evzzauxypt']
['mi', 'obsesi', 'on', 'adel', 'ser', 'a', 'busc', 'mecan', 'legal', 'derogaci', 'on', 'ley', 'estatutari', 'jep', 'asi', 'm', '…']
['a', 'ver', 'senador', 'urib', 'despu', 'es', '9', 'mes', 'perd', 'derrot', 'cam', 'derrot', 'sen', 'sentenci', 'q', '…']
['a', 'ver', 'senador', 'urib', 'despu', 'es', '9', 'mes', 'perd', 'derrot', 'cam', 'd

['que', 'pas', 'colombian', 'por', 'defiend', 'jep', 'justici', 'narcoterror', 'narcoterror', 'y', '…', 'https://t.co/7tgf2mxkwt']
['a', 'ver', 'senador', 'urib', 'despu', 'es', '9', 'mes', 'perd', 'derrot', 'cam', 'derrot', 'sen', 'sentenci', 'q', '…']
['pte', 'sient', 'derrot', 'hab', 'sancion', 'ley', 'estatutari', 'jep', 'al', 'contrari', 'sientat', 'tranquil', '…']
['no', 'escuch', 'o', 'mes', 'ias', 'pais', 'hoy', 'tard', 'su', 'objet', '1', 'a', 'acab', 'jep', '20', '…', 'https://t.co/2menaep4y5']
['cre', 'usted', 'jep', 'cort', 'cre', 'dar', 'impun', 'narcoguerriller', '…']
['president', 'duqu', 'despu', 'es', 'fracas', 'objecion', 'congres', 'cort', 'cumpl', 'mandat', 'constitu', '…']
['señor', 'onu', '1', 'uds', 'sab', 'ian', 'señor', 'numer', '82', 'list', 'dea', '2', 'uds', 'sab', 'facilit', 'proc', '…', 'https://t.co/fwrskospcx']
['ufff', 'casi', 'fin', 'habemus', 'ley', 'jep', 'https://t.co/ppv933eas']
['desesper', 'nivel', 'dios', 'no', 'sab', 'desaparec', 'jep', 'nadi',